# Explore Toronto Neighborhoods 

### Install request

In [2]:
!pip install bs4
#!pip install requests

In [3]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

### Scraping the web

In [4]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
data  = requests.get(url).text 

In [6]:
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'

In [7]:
import pandas as pd
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [15]:
df.head()

Borough                      Neighborhood Postal Code
0        North York                         Parkwoods         M3A
1        North York                  Victoria Village         M4A
2  Downtown Toronto         Regent Park, Harbourfront         M5A
3        North York  Lawrence Manor, Lawrence Heights         M6A
4      Queen's Park     Ontario Provincial Government         M7A

# Question 1

In [17]:
df.shape

(103, 3)

In [9]:
lat_lng_coords=pd.read_csv("E://GIthub clone//Github-clone//Capstone Project//Geospatial_Coordinates.csv")

In [16]:
lat_lng_coords.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

# Question 2

In [14]:
Toronto_df=df.merge(lat_lng_coords, how='left', on='Postal Code')
Toronto_df.head()

Borough                      Neighborhood Postal Code   Latitude  \
0        North York                         Parkwoods         M3A  43.753259   
1        North York                  Victoria Village         M4A  43.725882   
2  Downtown Toronto         Regent Park, Harbourfront         M5A  43.654260   
3        North York  Lawrence Manor, Lawrence Heights         M6A  43.718518   
4      Queen's Park     Ontario Provincial Government         M7A  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494

# Question 3

# Explore Toronto Downtown Borough

In [23]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


### Generate latitude and longitude of Toronto

In [24]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Visualize the neighborhood

In [26]:
# create map of Manhattan using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

###  Define Foursquare Credentials and Version

In [27]:
CLIENT_ID = 'YIEOT3TUNNYDPND435QCSD1AT03XXUH5QTYDHLXPP3ACHE0Q' # your Foursquare ID
CLIENT_SECRET = '30LQQXMH0X24WCIWQXPIN4ZGJOARWQFKZ2CFWDIO3KKW2C5E' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YIEOT3TUNNYDPND435QCSD1AT03XXUH5QTYDHLXPP3ACHE0Q
CLIENT_SECRET:30LQQXMH0X24WCIWQXPIN4ZGJOARWQFKZ2CFWDIO3KKW2C5E


# Explore Neighborhoods in Toronto

### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


### Let's check the size of the resulting dataframe

In [30]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1064, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0           Tandem Coffee       43.653559       -79.361809   
1        Roselle Desserts       43.653447       -79.362017   
2  Cooper Koo Family YMCA       43.653249       -79.358008   
3     Body Blitz Spa East       43.654735       -79.359874   
4          Impact Kitchen       43.656369       -79.356980   

        Venue Category  
0          Coffee Shop  
1               Bakery  
2  Distribution Center  
3                  Spa  
4           Restaurant

### Let's check how many venues were returned for each neighborhood

In [31]:
Toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            48   
CN Tower, King and Spadina, Railway Lands, Harb...                     18   
Central Bay Street                                                     64   
Christie                                                               15   
Church and Wellesley                                                   65   
Commerce Court, Victoria Hotel                                        100   
First Canadian Place, Underground city                                100   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
Kensington Market, Chinatown, Grange Park                              56   
Regent Park, Harbourfront                                              43   
Richmond, Adelaide, King                                              100   
Rosedale                                                                4   
St. James Town                                                         80   
St. James Town, Cabbagetown                                            36   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         35   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             48   
CN Tower, King and Spadina, Railway Lands, Harb...                      18   
Central Bay Street                                                      64   
Christie                                                                15   
Church and Wellesley                                                    65   
Commerce Court, Victoria Hotel                                         100   
First Canadian Place, Underground city                                 100   
Garden District, Ryerson                                               100   
Harbourfront East, Union Station, Toronto Islands                      100   
Kensington Market, Chinatown, Grange Park                               56   
Regent Park, Harbourfront                                               43   
Richmond, Adelaide, King                                               100   
Rosedale                                                                 4   
St. James Town                                                          80   
St. James Town, Cabbagetown                                             36   
Toronto Dominion Centre, Design Exchange                               100   
University of Toronto, Harbord                                          35   

                                                    Venue  Venue Latitude  \
Neighborhood                                                                
Berczy Park                                            48              48   
CN Tower, King and Spadina, Railway Lands, Harb...     18              18   
Central Bay Street                                     64              64   
Christie                                               15              15   
Church and Wellesley                                   65              65   
Commerce Court, Victoria Hotel                        100             100   
First Canadian Place, Underground city                100             100   
Garden District, Ryerson                              100             100   
Harbourfront East, Union Station, Toronto Islands     100             100   
Kensington Market, Chinatown, Grange Park              56              56   
Regent Park, Harbourfront                              43              43   
Richmond, Adelaide, King                              100             100   
Rosedale                   

### Let's find out how many unique categories can be curated from all the returned venues

In [32]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 184 uniques categories.


## Analyze Each Neighborhood

In [39]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

Yoga Studio  Adult Boutique  Afghan Restaurant  Airport  \
0            0               0                  0        0   
1            0               0                  0        0   
2            0               0                  0        0   
3            0               0                  0        0   
4            0               0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  ...  Theater  Theme Restaurant  \
0                 0                    0  ...        0                 0   
1                 0                    0  ...        0                 0   
2                 0                    0  ...        0                 0   
3                 0                    0  ...        0                 0   
4                 0                    0  ...        0                 0   

   Thrift / Vintage Store  Trail  Train Station  \
0                       0      0              0   
1                       0      0              0   
2                       0      0              0   
3                       0      0              0   
4                       0      0              0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                              0                 0                      0   
1                              0                 0                      0   
2                              0                 0                      0   
3                              0                 0                      0   
4                              0                 0                      0   

   Wine Bar  Wine Shop  
0         0          0  
1         0          0  
2         0          0  
3         0          0  
4         0          0  

[5 rows x 184 columns]

In [40]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

Neighborhood  Yoga Studio  \
0                                        Berczy Park     0.000000   
1  CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
2                                 Central Bay Street     0.015625   
3                                           Christie     0.000000   
4                               Church and Wellesley     0.015385   

   Adult Boutique  Afghan Restaurant   Airport  Airport Food Court  \
0        0.000000           0.000000  0.000000            0.000000   
1        0.000000           0.000000  0.055556            0.055556   
2        0.000000           0.000000  0.000000            0.000000   
3        0.000000           0.000000  0.000000            0.000000   
4        0.015385           0.015385  0.000000            0.000000   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  ...  \
0      0.000000        0.000000         0.000000          0.000000  ...   
1      0.055556        0.111111         0.166667          0.111111  ...   
2      0.000000        0.000000         0.000000          0.000000  ...   
3      0.000000        0.000000         0.000000          0.000000  ...   
4      0.000000        0.000000         0.000000          0.000000  ...   

    Theater  Theme Restaurant  Thrift / Vintage Store  Trail  Train Station  \
0  0.000000          0.000000                     0.0    0.0            0.0   
1  0.000000          0.000000                     0.0    0.0            0.0   
2  0.000000          0.000000                     0.0    0.0            0.0   
3  0.000000          0.000000                     0.0    0.0            0.0   
4  0.015385          0.015385                     0.0    0.0            0.0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                       0.041667               0.0                    0.0   
1                       0.000000               0.0                    0.0   
2                       0.015625               0.0                    0.0   
3                       0.000000               0.0                    0.0   
4                       0.000000               0.0                    0.0   

   Wine Bar  Wine Shop  
0  0.000000   0.000000  
1  0.000000   0.000000  
2  0.015625   0.000000  
3  0.000000   0.000000  
4  0.000000   0.015385  

[5 rows x 184 columns]

### Let's confirm the new size

In [42]:
Toronto_grouped.shape

(17, 184)

### Let's print each neighborhood along with the top 5 most common venues

In [43]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue  freq
0                   Cocktail Bar  0.06
1                         Bakery  0.06
2                 Sandwich Place  0.06
3                    Coffee Shop  0.06
4  Vegetarian / Vegan Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3       Coffee Shop  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1      Sandwich Place  0.08
2    Sushi Restaurant  0.06
3  Italian Restaurant  0.06
4                Café  0.05


----Christie----
           venue  freq
0  Grocery Store  0.27
1           Café  0.20
2           Park  0.13
3    Coffee Shop  0.07
4     Restaurant  0.07


----Church and Wellesley----
                 venue  freq
0     Sushi Restaurant  0.09
1  Japanese Restaurant  0.08
2

### Let's put that into a _pandas_ dataframe

##### First, let's write a function to sort the venues in descending order.

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [46]:
import numpy as np

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
2                                 Central Bay Street           Coffee Shop   
3                                           Christie         Grocery Store   
4                               Church and Wellesley      Sushi Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar                Bakery        Sandwich Place   
1        Airport Lounge      Airport Terminal           Coffee Shop   
2        Sandwich Place    Italian Restaurant      Sushi Restaurant   
3                  Café                  Park           Coffee Shop   
4   Japanese Restaurant            Restaurant               Gay Bar   

  5th Most Common Venue 6th Most Common Venue          7th Most Common Venue  \
0              Beer Bar    Seafood Restaurant  Vegetarian / Vegan Restaurant   
1       Harbor / Marina   Rental Car Location               Sculpture Garden   
2                  Café            Restaurant            Japanese Restaurant   
3             Nightclub            Restaurant             Italian Restaurant   
4           Coffee Shop         Burrito Place                            Pub   

  8th Most Common Venue 9th Most Common Venue    10th Most Common Venue  
0        Farmers Market            Restaurant                      Park  
1              Boutique                   Bar                     Plane  
2           Salad Place           Pizza Place                      Bank  
3            Baby Store    Athletics & Sports                  Creperie  
4     Indian Restaurant  Fast Food Restaurant  Mediterranean Restaurant

## Cluster Neighborhoods

In [48]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 2, 0, 3, 4, 0, 0, 0, 0, 4])

##### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

Borough               Neighborhood Postal Code   Latitude  \
0  Downtown Toronto  Regent Park, Harbourfront         M5A  43.654260   
1  Downtown Toronto   Garden District, Ryerson         M5B  43.657162   
2  Downtown Toronto             St. James Town         M5C  43.651494   
3  Downtown Toronto                Berczy Park         M5E  43.644771   
4  Downtown Toronto         Central Bay Street         M5G  43.657952   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.360636               0           Coffee Shop                  Park   
1 -79.378937               0           Coffee Shop        Clothing Store   
2 -79.375418               4           Coffee Shop          Cocktail Bar   
3 -79.373306               4           Coffee Shop          Cocktail Bar   
4 -79.387383               0           Coffee Shop        Sandwich Place   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                Bakery                   Pub                  Café   
1        Sandwich Place                  Café                 Hotel   
2                  Café    Italian Restaurant              Beer Bar   
3                Bakery        Sandwich Place              Beer Bar   
4    Italian Restaurant      Sushi Restaurant                  Café   

  6th Most Common Venue          7th Most Common Venue 8th Most Common Venue  \
0        Sandwich Place                     Beer Store            Restaurant   
1           Pizza Place                 Cosmetics Shop                  Bank   
2            Restaurant                 Clothing Store        Cosmetics Shop   
3    Seafood Restaurant  Vegetarian / Vegan Restaurant        Farmers Market   
4            Restaurant            Japanese Restaurant           Salad Place   

  9th Most Common Venue 10th Most Common Venue  
0        Breakfast Spot  Performing Arts Venue  
1   Japanese Restaurant     Italian Restaurant  
2   Japanese Restaurant    American Restaurant  
3            Restaurant                   Park  
4           Pizza Place                   Bank

##### Finally, let's visualize the resulting clusters

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [51]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels  \
0                           Regent Park, Harbourfront               0   
1                            Garden District, Ryerson               0   
4                                  Central Bay Street               0   
6                            Richmond, Adelaide, King               0   
7   Harbourfront East, Union Station, Toronto Islands               0   
8            Toronto Dominion Centre, Design Exchange               0   
9                      Commerce Court, Victoria Hotel               0   
10                     University of Toronto, Harbord               0   
14                        St. James Town, Cabbagetown               0   
15             First Canadian Place, Underground city               0   

   1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0            Coffee Shop                  Park                Bakery   
1            Coffee Shop        Clothing Store        Sandwich Place   
4            Coffee Shop        Sandwich Place    Italian Restaurant   
6            Coffee Shop                  Café        Sandwich Place   
7            Coffee Shop                  Café                 Hotel   
8            Coffee Shop                  Café                 Hotel   
9            Coffee Shop                  Café        Sandwich Place   
10                  Café           Coffee Shop        Sandwich Place   
14           Coffee Shop                  Café                Bakery   
15           Coffee Shop                  Café        Sandwich Place   

   4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0                    Pub                  Café        Sandwich Place   
1                   Café                 Hotel           Pizza Place   
4       Sushi Restaurant                  Café            Restaurant   
6                    Gym        Clothing Store            Restaurant   
7         Scenic Lookout           Pizza Place              Aquarium   
8         Sandwich Place      Asian Restaurant              Pharmacy   
9                  Hotel            Restaurant                   Gym   
10                   Bar   Japanese Restaurant                Bakery   
14            Restaurant                   Pub    Italian Restaurant   
15                 Hotel                   Gym   Japanese Restaurant   

   7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0             Beer Store            Restaurant        Breakfast Spot   
1         Cosmetics Shop                  Bank   Japanese Restaurant   
4    Japanese Restaurant           Salad Place           Pizza Place   
6       Sushi Restaurant                   Bar                Bakery   
7         Sandwich Place    Italian Restaurant   Sporting Goods Shop   
8             Restaurant           Salad Place                Bakery   
9    Japanese Restaurant      Asian Restaurant    Italian Restaurant   
10         Poutine Place              Beer Bar            Beer Store   
14           Pizza Place           Yoga Studio            Playground   
15         Deli / Bodega            Restaurant                  Bank   

   10th Most Common Venue  
0   Performing Arts Venue  
1      Italian Restaurant  
4                    Bank  
6        Asian Restaurant  
7              Sports Bar  
8           Deli / Bodega  
9           Deli / Bodega  
10             Restaurant  
14               Pharmacy  
15       Sushi Restaurant

#### Cluster 2

In [52]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
13     Rosedale               1                  Park                 Trail   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
13            Playground          Concert Hall        Discount Store   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
13                 Diner          Dessert Shop      Department Store   

   9th Most Common Venue 10th Most Common Venue  
13         Deli / Bodega           Dance Studio

#### Cluster 3

In [53]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels  \
12  CN Tower, King and Spadina, Railway Lands, Har...               2   

   1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
12       Airport Service        Airport Lounge      Airport Terminal   

   4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
12           Coffee Shop       Harbor / Marina   Rental Car Location   

   7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
12      Sculpture Garden              Boutique                   Bar   

   10th Most Common Venue  
12                  Plane

#### Cluster 4

In [54]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
5     Christie               3         Grocery Store                  Café   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
5                  Park           Coffee Shop             Nightclub   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
5            Restaurant    Italian Restaurant            Baby Store   

  9th Most Common Venue 10th Most Common Venue  
5    Athletics & Sports               Creperie

#### Cluster 5

In [55]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels  \
2                              St. James Town               4   
3                                 Berczy Park               4   
11  Kensington Market, Chinatown, Grange Park               4   
16                       Church and Wellesley               4   

   1st Most Common Venue          2nd Most Common Venue 3rd Most Common Venue  \
2            Coffee Shop                   Cocktail Bar                  Café   
3            Coffee Shop                   Cocktail Bar                Bakery   
11                  Café  Vegetarian / Vegan Restaurant           Coffee Shop   
16      Sushi Restaurant            Japanese Restaurant            Restaurant   

   4th Most Common Venue 5th Most Common Venue  6th Most Common Venue  \
2     Italian Restaurant              Beer Bar             Restaurant   
3         Sandwich Place              Beer Bar     Seafood Restaurant   
11                Bakery    Mexican Restaurant  Vietnamese Restaurant   
16               Gay Bar           Coffee Shop          Burrito Place   

            7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
2                  Clothing Store        Cosmetics Shop   Japanese Restaurant   
3   Vegetarian / Vegan Restaurant        Farmers Market            Restaurant   
11                    Gaming Cafe            Food Truck          Burger Joint   
16                            Pub     Indian Restaurant  Fast Food Restaurant   

      10th Most Common Venue  
2        American Restaurant  
3                       Park  
11                Food Court  
16  Mediterranean Restaurant